In [1]:
import pandas as pd
from Main import settings, features, pf_set
import time
import pickle
from pandas.tseries.offsets import DateOffset
import Prepare_Data
import data_run_files
import return_prediction_functions
from return_prediction_functions import rff
import numpy as np
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import Ridge
import General_Functions
import pickle
import Estimate_Covariance_Matrix
import os
import prepare_portfolio_data
import portfolio_choice_functions
from itertools import product

start_date = pd.to_datetime('1952-12-31')

#Relevante paths
output_path = "./data_fifty/"
pickle_file_path = "./data_fifty/model_1.pkl"
wealth_path = "./data_fifty/wealth.csv"
barra_file_path = "./data_fifty/barra_cov.pkl"
output_path_market_returns = "data_fifty/market_returns_test.csv"

risk_free_path = "./data_fifty/risk_free_test.csv" 
output_data_ret_csv = "./data_fifty/data_ret.csv"  
output_data_ret_ld1_csv = "./data_fifty/data_ret_ld1.csv"

file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"

risk_free = data_run_files.process_risk_free_rate(risk_free_path, start_date)
daily_path = "./data_fifty/daily.csv"
chars_path = "./data_fifty/chars_behandlet.parquet"


Features list length: 154
Filtrering udført. Antal rækker efter filtrering: 841
Fil gemt som ./data_test/risk_free_test.csv


# Indlæser wealth, chars, daily, data_ret og data_ret_ld1

In [2]:
wealth = pd.read_csv(wealth_path)
wealth['eom'] = pd.to_datetime(wealth['eom'])
data_ret = pd.read_csv(output_data_ret_csv)
data_ret_ld1 = pd.read_csv(output_data_ret_ld1_csv)
data_ret['eom'] = pd.to_datetime(data_ret['eom'])
data_ret_ld1['eom'] = pd.to_datetime(data_ret_ld1['eom'])
data_ret_ld1['eom_ret'] = pd.to_datetime(data_ret_ld1['eom_ret'])
daily = pd.read_csv(daily_path, parse_dates=["date", "eom"])
chars = pd.read_parquet(chars_path)

Disse er checket og se ser korrekte ud wuhu :D

In [3]:
data_ret

,id,eom,ret_ld1,ret_ld2,ret_ld3,ret_ld4,ret_ld5,ret_ld6,ret_ld7,ret_ld8,ret_ld9,ret_ld10,ret_ld11,ret_ld12
0,10104,1986-04-30,-0.064934,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897
1,10104,1986-05-31,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814
2,10104,1986-06-30,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517
3,10104,1986-07-31,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414
4,10104,1986-08-31,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414,0.171080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90698,92655,2022-07-31,-0.044328,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90699,92655,2022-08-31,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90700,92655,2022-09-30,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90701,92655,2022-10-31,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Hent Predictions :D

### Indlæsning af Pickle fil (predictions)

In [4]:
with open(pickle_file_path, "rb") as file:
    model_1 = pickle.load(file)

#print(model_1) #Vi bruger dog ikke ikke denne i selve 

# Barra COV

Just a quick check

In [5]:
with open(barra_file_path, "rb") as file:
    barra_cov = pickle.load(file)

In [6]:
print(barra_cov['2022-11-30'])

{'fct_load':        BusEq  Chems  Durbl  Enrgy  Hlth  Manuf  Money  NoDur  Other  Shops  \
id                                                                           
10104    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10107    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10138    0.0    0.0    0.0    0.0   0.0    0.0    1.0    0.0    0.0    0.0   
10696    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
11308    0.0    0.0    0.0    0.0   0.0    0.0    0.0    1.0    0.0    0.0   
...      ...    ...    ...    ...   ...    ...    ...    ...    ...    ...   
87299    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87432    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87447    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
87717    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
92655    0.0    0.0    0.0    0.0   0.0    0.0    1

In [7]:
print(barra_cov['2022-11-30']['ivol_vec'])

id
10104    0.004043
10107    0.002393
10138    0.002996
10696    0.003696
11308    0.001280
           ...   
87299    0.004106
87432    0.003534
87447    0.003139
87717    0.003166
92655    0.002555
Name: res_vol, Length: 180, dtype: float64


# Prepare Portfolio Data - tilføj return predictions, lambda list og datoer

In [8]:
chars = prepare_portfolio_data.add_return_predictions(chars,settings, output_path)

In [9]:
print(chars.shape) #Skal være 73216 x 131
print(chars[['id', 'eom','pred_ld1']])

(73216, 131)
          id        eom  pred_ld1
0      10104 1986-12-31       NaN
1      10104 1987-01-31       NaN
2      10104 1987-02-28       NaN
3      10104 1987-03-31       NaN
4      10104 1987-04-30       NaN
...      ...        ...       ...
73211  92655 2022-07-31  0.010957
73212  92655 2022-08-31  0.009506
73213  92655 2022-09-30  0.010641
73214  92655 2022-10-31  0.010774
73215  92655 2022-11-30  0.011608

[73216 rows x 3 columns]


In [10]:
lambda_list = prepare_portfolio_data.create_lambda_list(chars)

In [11]:
print(lambda_list['2021-10-31'])

{10104: 2.1068747835833751e-10, 10107: 3.034783408033585e-11, 10138: 1.014425423141795e-09, 10696: 4.3268657868458776e-10, 11308: 2.5674127083269843e-10, 11850: 1.5210112818417197e-10, 11955: 9.846108938145744e-10, 12060: 2.392171855561806e-10, 12062: 1.1115228863141125e-09, 12490: 3.103196145302579e-10, 13688: 1.2451492409136745e-09, 13856: 3.0647710531490904e-10, 13901: 5.984566822586187e-10, 14008: 3.4654779374858333e-10, 14277: 5.572628200703795e-10, 14541: 1.8678925436727748e-10, 14593: 1.7957730786929064e-11, 14702: 1.9775531529792074e-10, 15720: 1.868244462399027e-09, 17778: 3.3642773554010653e-10, 18163: 1.9283146734748652e-10, 18411: 7.948401354253297e-10, 19393: 3.2583660018022594e-10, 19561: 8.240904270111589e-11, 20482: 3.1099118409527434e-10, 21371: 1.314659590664529e-09, 21936: 1.611785023727204e-10, 22103: 8.191041680175024e-10, 22111: 1.9039009050878924e-10, 22592: 4.542098255993858e-10, 22752: 2.035214983043305e-10, 25320: 1.6935230409360711e-09, 25785: 1.9152306625614

In [60]:
first_cov_date, hp_years, start_oos = prepare_portfolio_data.define_important_dates(barra_cov, settings)
date_ranges = prepare_portfolio_data.create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]
#Disse er alle korrekte

# Base cases

In [13]:
ew = portfolio_choice_functions.ew_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set=pf_set)
mkt = portfolio_choice_functions.mkt_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
rw = portfolio_choice_functions.rw_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
mv = portfolio_choice_functions.mv_implement(data=chars, cov_list=barra_cov ,wealth=wealth, dates=dates_oos, pf_set= pf_set)
factor_ml = portfolio_choice_functions.factor_ml_implement(data = chars, wealth = wealth, dates= dates_oos, n_pfs=settings["factor_ml"]["n_pfs"]
, gam=pf_set['gamma_rel'])

In [14]:
ew

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.005556       NaN       NaN
 1      10104 2011-01-31  0.005582  0.005556  0.024954  0.020052
 2      10104 2011-02-28  0.005493  0.005556  0.027140  0.038821
 3      10104 2011-03-31  0.005627  0.005556  0.016218  0.003383
 4      10104 2011-04-30  0.005825  0.005556  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.005393  0.005587  0.055684  0.093630
 25860  92655 2022-08-31  0.005544  0.005587 -0.043528 -0.036188
 25861  92655 2022-09-30  0.005999  0.005587 -0.024368 -0.091370
 25862  92655 2022-10-31  0.005677  0.005556  0.098793  0.081350
 25863  92655 2022-11-30  0.005211  0.005556 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret type
 0    1.0       0.0  0.848747  0.022582  0.006040 2011-01-31  1/N
 1    1.0       0.0  0.052598  0.037496  0.00005

In [15]:
mkt

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.019620       NaN       NaN
 1      10104 2011-01-31  0.019714  0.019548  0.024954  0.020052
 2      10104 2011-02-28  0.019328  0.019582  0.027140  0.038821
 3      10104 2011-03-31  0.019833  0.020010  0.016218  0.003383
 4      10104 2011-04-30  0.020979  0.021030  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.023291  0.023322  0.055684  0.093630
 25860  92655 2022-08-31  0.023144  0.023286 -0.043528 -0.036188
 25861  92655 2022-09-30  0.025003  0.024959 -0.024368 -0.091370
 25862  92655 2022-10-31  0.025361  0.025229  0.098793  0.081350
 25863  92655 2022-11-30  0.023664  0.023718 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r            tc    eom_ret    type
 0    1.0       0.0  0.000000  0.023370  0.000000e+00 2011-01-31  Market
 1    1.0       0.0  0.013623  0.0

In [16]:
rw

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.000123       NaN       NaN
 1      10104 2011-01-31  0.000124 -0.007284  0.024954  0.020052
 2      10104 2011-02-28 -0.007202  0.001111  0.027140  0.038821
 3      10104 2011-03-31  0.001125 -0.010000  0.016218  0.003383
 4      10104 2011-04-30 -0.010484 -0.008272  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31 -0.008677 -0.005243  0.055684  0.093630
 25860  92655 2022-08-31 -0.005204 -0.011486 -0.043528 -0.036188
 25861  92655 2022-09-30 -0.012333 -0.006242 -0.024368 -0.091370
 25862  92655 2022-10-31 -0.006343 -0.006296  0.098793  0.081350
 25863  92655 2022-11-30 -0.005906 -0.001605 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret     type
 0    2.0       1.0  2.519567  0.015278  0.077693 2011-01-31  Rank-ML
 1    2.0       1.0  0.826170  0.006433 

In [17]:
mv

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.017280       NaN       NaN
 1      10104 2011-01-31  0.017363  0.022978  0.024954  0.020052
 2      10104 2011-02-28  0.022719  0.019329  0.027140  0.038821
 3      10104 2011-03-31  0.019576  0.014793  0.016218  0.003383
 4      10104 2011-04-30  0.015510  0.024127  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31 -0.014596 -0.010913  0.055684  0.093630
 25860  92655 2022-08-31 -0.010830 -0.006496 -0.043528 -0.036188
 25861  92655 2022-09-30 -0.006975  0.005305 -0.024368 -0.091370
 25862  92655 2022-10-31  0.005390 -0.000461  0.098793  0.081350
 25863  92655 2022-11-30 -0.000433 -0.008272 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':           inv  shorting  turnover         r        tc    eom_ret  \
 0    2.439181  0.719590  2.475996 -0.038908  0.103482 2011-01-31   
 1    2.460849  0.730424  1.173957  0.0454

In [18]:
factor_ml

{'w':           id        eom  w_start    w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.01962  0.0       NaN       NaN
 1      10104 2011-01-31  0.00000  0.0  0.024954  0.020052
 2      10104 2011-02-28  0.00000  0.0  0.027140  0.038821
 3      10104 2011-03-31  0.00000  0.0  0.016218  0.003383
 4      10104 2011-04-30  0.00000  0.0  0.077442  0.027662
 ...      ...        ...      ...  ...       ...       ...
 25859  92655 2022-07-31  0.00000  0.0  0.055684  0.093630
 25860  92655 2022-08-31  0.00000  0.0 -0.043528 -0.036188
 25861  92655 2022-09-30  0.00000  0.0 -0.024368 -0.091370
 25862  92655 2022-10-31  0.00000  0.0  0.098793  0.081350
 25863  92655 2022-11-30  0.00000  0.0 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret       type
 0    2.0       1.0  2.924576  0.017222  0.154048 2011-01-31  Factor-ML
 1    2.0       1.0  1.022224  0.018595  0.074021 2011-02-28  Factor-ML
 2    2.0       1.0  1.455416  0.009

In [19]:
tpf = portfolio_choice_functions.tpf_implement(data=chars, cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set["gamma_rel"])

In [20]:
tpf

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.015546       NaN       NaN
 1      10104 2011-01-31  0.015621  0.017275  0.024954  0.020052
 2      10104 2011-02-28  0.017081  0.022007  0.027140  0.038821
 3      10104 2011-03-31  0.022288  0.006919  0.016218  0.003383
 4      10104 2011-04-30  0.007254  0.017548  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.007726  0.034372  0.055684  0.093630
 25860  92655 2022-08-31  0.034111  0.024339 -0.043528 -0.036188
 25861  92655 2022-09-30  0.026134  0.075025 -0.024368 -0.091370
 25862  92655 2022-10-31  0.076235  0.059830  0.098793  0.081350
 25863  92655 2022-11-30  0.056120  0.040740 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':           inv  shorting  turnover         r        tc    eom_ret          type
 0    2.374874  0.911181  2.749611 -0.019087  0.114453 2011-01-31  Markowitz-ML
 1    2.352451  0.88

# STATIC ML

In [68]:
import pandas as pd
import numpy as np
import itertools

# Eksempel: De brugerdefinerede funktioner skal defineres et andet sted
# def initial_weights_new(data, w_type): ...
# def create_cov(cov_item, ids): ...
# def sigma_gam_adj(sigma_gam, g, cov_type): ...
# def create_lambda(lambda_item, ids): ...
# def pf_ts_fun(data, **kwargs): ...

def static_val_fun(data, dates, cov_list, lambda_list, wealth, cov_type, gamma_rel, k=None, g=None, u=None, hps=None):
    """
    Beregner de statiske vægte for de angivne datoer.
    Parametre:
      - data: DataFrame med portefølje-data.
      - dates: Liste af datoer (som datetime-objekter).
      - cov_list: Dictionary med covariance-data pr. dato.
      - lambda_list: Dictionary med lambda-data pr. dato.
      - wealth: DataFrame med wealth-information (skal indeholde 'eom' og 'wealth', evt. 'mu_ld1').
      - cov_type: Type til justering af covariance.
      - gamma_rel: Skalar til vægtning.
      - k, g, u: Parametre, kan angives direkte.
      - hps: DataFrame med hyperparametre; hvis angivet, opdateres k, g og u for hver dato.
    """
    print("Nøgler i cov_list:", list(cov_list.keys()))
    print("Nøgler i lambda_list:", list(lambda_list.keys()))
    # Start med initiale weights (f.eks. VW, alt efter initial_weights_new)
    static_weights = General_Functions.initial_weights_new(data, w_type="vw")
    
    # Merge med kolonner fra data: id, eom, tr_ld1 og pred_ld1
    static_weights = static_weights.merge(data[['id', 'eom', 'tr_ld1', 'pred_ld1']], on=['id', 'eom'], how='left')
    # Merge med wealth for at få mu_ld1
    static_weights = static_weights.merge(wealth[['eom', 'mu_ld1']], on='eom', how='left')
    
    for d in dates:
        print("Processing date:", d)
        mask = static_weights['eom'] == d
        print("Antal rækker for dato {}: {}".format(d, mask.sum()))
        print("Data for dato {}:".format(d), static_weights.loc[mask].head())
        # Hvis hyperparametre (hps) er angivet, så opdater k, g og u
        if hps is not None:
            # Filtrer rækker hvor året for eom_ret er mindre end året for d
            hp_candidates = hps[hps['eom_ret'].dt.year < d.year]
            if not hp_candidates.empty:
                # Vælg den række med den seneste eom_ret
                hp = hp_candidates.loc[hp_candidates['eom_ret'].idxmax()]
                g = hp['g']
                u = hp['u']
                k = hp['k']
        
        # Hent wealth for datoen
        wealth_t_series = wealth.loc[wealth['eom'] == d, 'wealth']
        print("Wealth for dato {}: {}".format(d, wealth_t_series))
        if wealth_t_series.empty:
            print("Ingen wealth data for dato:", d)
            continue  # Eller håndtér fejlen, hvis der mangler data for d
        wealth_t = wealth_t_series.iloc[0]
        print("wealth_t:", wealth_t)
        
        # Vælg de rækker i static_weights for datoen d
        mask = static_weights['eom'] == d
        # Sørg for, at id'erne er strenge (for at matche med de oprettede matricer)
        ids = static_weights.loc[mask, 'id'].astype(str)

        print(d)
        # Byg covariance matrix og tilpas den
        sigma_gam = General_Functions.create_cov(cov_list[d], ids=ids.tolist()) * gamma_rel
        print("sigma_gam (før justering):", sigma_gam)
        sigma_gam = General_Functions.sigma_gam_adj(sigma_gam, g=g, cov_type=cov_type)
        print("sigma_gam (efter justering):", sigma_gam)
        
        # Byg lambda matrix og skaler med k
        lambda_val = General_Functions.create_lambda(lambda_list[d], ids=ids.tolist())
        print("lambda (før skalering):", lambda_val)
        lambda_val = lambda_val * k
        print("lambda (efter skalering):", lambda_val)
        
        # Ekstraher de nødvendige kolonner som numpy-arrays
        pred_ld1 = static_weights.loc[mask, 'pred_ld1'].values
        w_start = static_weights.loc[mask, 'w_start'].values
        
        # Beregn højresidevektoren: pred_ld1 * u + wealth_t * (lambda @ w_start)
        print("pred_ld1:", pred_ld1)
        print("w_start:", w_start)
        rhs = (pred_ld1 * u) + wealth_t * (lambda_val.dot(w_start))
        print("rhs:", rhs)
        # Matrix A: sigma_gam + wealth_t * lambda
        A = sigma_gam + wealth_t * lambda_val
        print("A:", A)
        # Løs lineært system for at få nye vægte
        w_new = np.linalg.solve(A, rhs)
        print("w_new:", w_new)
        # Tildel de beregnede vægte til kolonnen 'w' for de relevante rækker
        static_weights.loc[mask, 'w'] = w_new
        
        # Opdater startvægt for næste måned:
        try:
            current_index = dates.index(d)
            next_month = dates[current_index + 1]
        except IndexError:
            # Hvis der ikke er nogen næste dato, springes opdateringen over
            continue
        
        # Udregn 'w_opt_lag1' for de rækker, der svarer til d, men til næste måned
        temp = static_weights.loc[mask, ['id', 'w', 'tr_ld1', 'mu_ld1']].copy()
        temp['eom'] = next_month
        temp['w_opt_lag1'] = temp['w'] * (1 + temp['tr_ld1']) / (1 + temp['mu_ld1'])
        
        # Merge temp med static_weights på ['id', 'eom']
        static_weights = static_weights.merge(temp[['id', 'eom', 'w_opt_lag1']], on=['id', 'eom'], how='left')
        
        # Sæt w_start for rækker med eom == next_month
        next_mask = static_weights['eom'] == next_month
        static_weights.loc[next_mask, 'w_start'] = static_weights.loc[next_mask, 'w_opt_lag1']
        static_weights.loc[next_mask & static_weights['w_start'].isna(), 'w_start'] = 0
        
        # Fjern den midlertidige kolonne
        static_weights.drop(columns='w_opt_lag1', inplace=True)
        
    return static_weights


In [69]:
def static_implement(data_tc, cov_list, lambda_list, rf, wealth, mu, gamma_rel,
                     dates_full, dates_oos, dates_hp, hp_years,
                     k_vec, u_vec, g_vec, cov_type, validation=None, seed=None):
    """
    Fuldt statisk implementering.
    Parametre:
      - data_tc: DataFrame med tidsserie-data.
      - cov_list, lambda_list: Ordbøger med covariance- og lambda-data.
      - rf, wealth, mu, gamma_rel: Diverse parametre.
      - dates_full, dates_oos, dates_hp: Lister af datoer (skal være datetime-objekter).
      - hp_years: Årstal for HP-valg.
      - k_vec, u_vec, g_vec: Vektorer med hyperparametre.
      - cov_type: Typen af covariance.
      - validation: Valideringsresultater (kan være None, hvorefter de udregnes).
      - seed: Til eventuel reproducibilitet.
    """
    # Opret HP-grid: alle kombinationer af k, u og g
    static_hps = pd.DataFrame(list(itertools.product(k_vec, u_vec, g_vec)), columns=['k', 'u', 'g'])
    
    # Udvælg relevant data: valid == True og eom i dates_hp, med kolonnerne id, eom, me, tr_ld1, pred_ld1, sorteret efter id og eom
    data_rel = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_hp))][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    data_rel = data_rel.sort_values(by=['id', 'eom'])
    
    # Hvis der ikke allerede er beregnet validation, udregnes den
    if validation is None:
        validation_list = []
        for i in range(len(static_hps)):
            print(i)
            hp = static_hps.iloc[i]
            static_w = static_val_fun(data_rel, dates=dates_hp, cov_list=cov_list, lambda_list=lambda_list,
                                      wealth=wealth, gamma_rel=gamma_rel, k=hp['k'], g=hp['g'], u=hp['u'], cov_type=cov_type)
            # Anvend pf_ts_fun på resultatet – denne funktion skal returnere en DataFrame med bl.a. 'eom_ret', 'r' og 'tc'
            static_w_pf = General_Functions.pf_ts_fun(static_w, data=data_tc, wealth=wealth, gam=gamma_rel)
            static_w_pf['hp_no'] = i
            static_w_pf['k'] = hp['k']
            static_w_pf['g'] = hp['g']
            static_w_pf['u'] = hp['u']
            validation_list.append(static_w_pf)
        validation = pd.concat(validation_list, ignore_index=True)
    
    # Sortér validation på hp_no og eom_ret
    validation.sort_values(by=['hp_no', 'eom_ret'], inplace=True)
    
    # Beregn kumulative middelværdier (cummean) per hp_no
    # Først: beregn kum. middel for r^2 og r
    validation['cum_mean_r2'] = validation.groupby('hp_no')['r'].transform(lambda x: x.pow(2).expanding().mean())
    validation['cum_mean_r']  = validation.groupby('hp_no')['r'].transform(lambda x: x.expanding().mean())
    validation['cum_var'] = validation['cum_mean_r2'] - validation['cum_mean_r'] ** 2
    
    # Beregn cum_obj for hver hp_no: cummean(r - tc - 0.5 * cum_var * gamma_rel)
    def compute_cum_obj(group):
        return (group['r'] - group['tc'] - 0.5 * group['cum_var'] * gamma_rel).expanding().mean()
    
    validation['cum_obj'] = validation.groupby('hp_no', group_keys=False).apply(compute_cum_obj)
    
    # Beregn rangering inden for hver eom_ret-gruppe baseret på -cum_obj
    validation['rank'] = validation.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='first')
    
    # Plottekoden er fjernet som ønsket.
    
    # Vælg de optimale HP'er: de rækker hvor eom_ret har måned == 12 og rank == 1, sorteret efter eom_ret
    optimal_hps = validation[(validation['eom_ret'].dt.month == 12) & (validation['rank'] == 1)].sort_values(by='eom_ret')
    
    # Implementér den endelige portefølje: vælg data_tc hvor eom i dates_oos og valid == True
    w_data = data_tc[(data_tc['eom'].isin(dates_oos)) & (data_tc['valid'] == True)][['id', 'eom', 'me', 'tr_ld1', 'pred_ld1']]
    w = static_val_fun(w_data, dates=dates_oos, cov_list=cov_list, lambda_list=lambda_list,
                        wealth=wealth, gamma_rel=gamma_rel, hps=optimal_hps, cov_type=cov_type)
    
    pf = General_Functions.pf_ts_fun(w, data=data_tc, wealth=wealth, gam=gamma_rel)
    pf['type'] = "Static-ML*"
    
    return {"hps": validation, "best_hps": optimal_hps, "w": w, "pf": pf}

In [34]:
dates_hp

DatetimeIndex(['2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31',
               '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31',
               '2001-08-31', '2001-09-30',
               ...
               '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
               '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30'],
              dtype='datetime64[ns]', length=264, freq='ME')

In [70]:
static = static_implement(
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    rf=risk_free,              # Data
    wealth=wealth,
    mu=pf_set['mu'],
    gamma_rel=pf_set['gamma_rel'],  # Investor
    dates_full=dates_m1,
    dates_oos=dates_oos,
    dates_hp=dates_hp,
    hp_years=hp_years,          # Dates
    k_vec=settings['pf']['hps']['static']['k'],
    u_vec=settings['pf']['hps']['static']['u'],
    g_vec=settings['pf']['hps']['static']['g'],
    cov_type=settings['pf']['hps']['cov_type'],
    validation=None
)

C:\Users\jonas\AppData\Local\Temp\ipykernel_15328\534823635.py:21: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.


0
Nøgler i cov_list: ['1994-10-31', '1994-11-30', '1994-12-31', '1995-01-31', '1995-02-28', '1995-03-31', '1995-04-30', '1995-05-31', '1995-06-30', '1995-07-31', '1995-08-31', '1995-09-30', '1995-10-31', '1995-11-30', '1995-12-31', '1996-01-31', '1996-02-29', '1996-03-31', '1996-04-30', '1996-05-31', '1996-06-30', '1996-07-31', '1996-08-31', '1996-09-30', '1996-10-31', '1996-11-30', '1996-12-31', '1997-01-31', '1997-02-28', '1997-03-31', '1997-04-30', '1997-05-31', '1997-06-30', '1997-07-31', '1997-08-31', '1997-09-30', '1997-10-31', '1997-11-30', '1997-12-31', '1998-01-31', '1998-02-28', '1998-03-31', '1998-04-30', '1998-05-31', '1998-06-30', '1998-07-31', '1998-08-31', '1998-09-30', '1998-10-31', '1998-11-30', '1998-12-31', '1999-01-31', '1999-02-28', '1999-03-31', '1999-04-30', '1999-05-31', '1999-06-30', '1999-07-31', '1999-08-31', '1999-09-30', '1999-10-31', '1999-11-30', '1999-12-31', '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31'

C:\Users\jonas\AppData\Local\Temp\ipykernel_15328\534823635.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
C:\Users\jonas\AppData\Local\Temp\ipykernel_15328\534823635.py:65: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.


Nøgler i cov_list: ['1994-10-31', '1994-11-30', '1994-12-31', '1995-01-31', '1995-02-28', '1995-03-31', '1995-04-30', '1995-05-31', '1995-06-30', '1995-07-31', '1995-08-31', '1995-09-30', '1995-10-31', '1995-11-30', '1995-12-31', '1996-01-31', '1996-02-29', '1996-03-31', '1996-04-30', '1996-05-31', '1996-06-30', '1996-07-31', '1996-08-31', '1996-09-30', '1996-10-31', '1996-11-30', '1996-12-31', '1997-01-31', '1997-02-28', '1997-03-31', '1997-04-30', '1997-05-31', '1997-06-30', '1997-07-31', '1997-08-31', '1997-09-30', '1997-10-31', '1997-11-30', '1997-12-31', '1998-01-31', '1998-02-28', '1998-03-31', '1998-04-30', '1998-05-31', '1998-06-30', '1998-07-31', '1998-08-31', '1998-09-30', '1998-10-31', '1998-11-30', '1998-12-31', '1999-01-31', '1999-02-28', '1999-03-31', '1999-04-30', '1999-05-31', '1999-06-30', '1999-07-31', '1999-08-31', '1999-09-30', '1999-10-31', '1999-11-30', '1999-12-31', '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', 

In [64]:
static

{'hps':       inv  shorting  turnover    r   tc    eom_ret  hp_no    k    g     u  \
 0     0.0       0.0       0.0  0.0  0.0 2001-01-31      0  1.0  0.0  0.25   
 1     0.0       0.0       0.0  0.0  0.0 2001-02-28      0  1.0  0.0  0.25   
 2     0.0       0.0       0.0  0.0  0.0 2001-03-31      0  1.0  0.0  0.25   
 3     0.0       0.0       0.0  0.0  0.0 2001-04-30      0  1.0  0.0  0.25   
 4     0.0       0.0       0.0  0.0  0.0 2001-05-31      0  1.0  0.0  0.25   
 ...   ...       ...       ...  ...  ...        ...    ...  ...  ...   ...   
 7123  0.0       0.0       0.0  0.0  0.0 2022-08-31     26  0.2  2.0  1.00   
 7124  0.0       0.0       0.0  0.0  0.0 2022-09-30     26  0.2  2.0  1.00   
 7125  0.0       0.0       0.0  0.0  0.0 2022-10-31     26  0.2  2.0  1.00   
 7126  0.0       0.0       0.0  0.0  0.0 2022-11-30     26  0.2  2.0  1.00   
 7127  0.0       0.0       0.0  0.0  0.0 2022-12-31     26  0.2  2.0  1.00   
 
       cum_mean_r2  cum_mean_r  cum_var  cum_obj  rank 

In [62]:
print(dates_m1)
print(dates_m2)
print(dates_oos)
print(dates_hp)

[datetime.date(2000, 12, 31), datetime.date(2001, 1, 31), datetime.date(2001, 2, 28), datetime.date(2001, 3, 31), datetime.date(2001, 4, 30), datetime.date(2001, 5, 31), datetime.date(2001, 6, 30), datetime.date(2001, 7, 31), datetime.date(2001, 8, 31), datetime.date(2001, 9, 30), datetime.date(2001, 10, 31), datetime.date(2001, 11, 30), datetime.date(2001, 12, 31), datetime.date(2002, 1, 31), datetime.date(2002, 2, 28), datetime.date(2002, 3, 31), datetime.date(2002, 4, 30), datetime.date(2002, 5, 31), datetime.date(2002, 6, 30), datetime.date(2002, 7, 31), datetime.date(2002, 8, 31), datetime.date(2002, 9, 30), datetime.date(2002, 10, 31), datetime.date(2002, 11, 30), datetime.date(2002, 12, 31), datetime.date(2003, 1, 31), datetime.date(2003, 2, 28), datetime.date(2003, 3, 31), datetime.date(2003, 4, 30), datetime.date(2003, 5, 31), datetime.date(2003, 6, 30), datetime.date(2003, 7, 31), datetime.date(2003, 8, 31), datetime.date(2003, 9, 30), datetime.date(2003, 10, 31), datetime.da

In [61]:
#dates_hp = list(dates_hp.to_pydatetime())
dates_m1 = [d.date() for d in dates_m1]
dates_m2 = [d.date() for d in dates_m2]
dates_oos = [d.date() for d in dates_oos]
dates_hp = [d.date() for d in dates_hp]
#dates_hp = list(dates_hp.to_pydatetime())
#dates_hp = list(dates_hp.to_pydatetime())

In [43]:
hp_years

[2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

# Predictions og Barra Cov (Ikke kør denne del)

Predictions

In [ ]:
search_grid_single = pd.DataFrame({
    'name': ['m1'],
    'horizon': [1]
})
search_grid = search_grid_single

In [ ]:
start_time = time.time()
models = []  # Liste til at gemme output for hver horizon

# Iterer over rækkerne i search_grid
for i in range(len(search_grid)):
    # Forbered y-variablen:
    h = search_grid.iloc[i]["horizon"]  # fx 1
    # Konstruer kolonnenavnet, fx "ret_ld1"
    col_name = "ret_ld" + str(h)
    # Beregn gennemsnittet pr. række – hvis der kun er én kolonne, svarer det til kolonneværdierne
    pred_y_values = data_ret[col_name]  # antages at være en Pandas Series
    # Opret en DataFrame med de nødvendige kolonner: id, eom
    pred_y_df = data_ret[['id', 'eom']].copy()
    # Beregn eom_pred_last: Vi antager, at vi ønsker at beregne slutdatoen for forudsigelsen som:
    # eom + (h+1) måneder - 1 dag (svarer nogenlunde til R's eom+1+months(max(h))-1)
    pred_y_df['eom_pred_last'] = pred_y_df['eom'] + MonthEnd(1)
    pred_y_df['ret_pred'] = pred_y_values

    # Join med de rækker i 'chars', hvor valid er True, på kolonnerne id og eom
    valid_chars = chars[chars['valid'] == True]
    data_pred = pd.merge(pred_y_df, valid_chars, on=['id', 'eom'], how='inner')
    print("data_pred.head")
    print(data_pred.head())
    print("data_pred.tail")
    print(data_pred.tail())
    print("horizons:", [h])

    # Bestem validerings-slutdatoer og test-inc baseret på settings
    update_freq = settings['split']['model_update_freq']
    if update_freq == "once":
        val_ends = [settings['split']['train_end']]
        test_inc = 1000
    elif update_freq == "yearly":
        # Opret en liste af datoer med årligt interval
        val_ends = pd.date_range(start=settings['split']['train_end'],
                                 end=settings['split']['test_end'],
                                 freq='YE').to_pydatetime().tolist()
        test_inc = 1
    elif update_freq == "decade":
        start_date = pd.to_datetime(settings['split']['train_end'])
        end_date = pd.to_datetime(settings['split']['test_end'])
        val_ends = []
        current = start_date
        while current <= end_date:
            val_ends.append(current)
            current += pd.DateOffset(years=10)
        test_inc = 10
    else:
        raise ValueError("Ugyldig model_update_freq i settings.")

    op = {}  # Dictionary til at gemme modeloutput for hver val_end
    inner_start = time.time()
    # Iterer over hver validerings-slutdato
    for val_end in val_ends:
        print(val_end)
        train_test_val = return_prediction_functions.data_split(
            data_pred,
            type=update_freq,
            val_end=val_end,
            val_years=settings['split']['val_years'],
            train_start=settings['screens']['start'],
            train_lookback=settings['split']['train_lookback'],
            retrain_lookback=settings['split']['retrain_lookback'],
            test_inc=test_inc,
            test_end=settings['split']['test_end']
        )
        print("datasættet er tomt; hvis ja, stop loopet")
        if train_test_val["test"].empty:
            print("Test datasættet er tomt for valideringsperiode:", val_end, ". Stopper forudsigelser.")
            break
        print("Train data:")
        print(train_test_val["train"].head())
        print("\nValidation data:")
        print(train_test_val["val"].head())
        print("\nTrain full data:")
        print(train_test_val["train_full"].head())
        print("\nTest data:")
        print(train_test_val["test"].head())
        model_start = time.time()
        model_op = return_prediction_functions.rff_hp_search(
            train_test_val,
            feat=features,
            p_vec=settings['rff']['p_vec'],
            g_vec=settings['rff']['g_vec'],
            l_vec=settings['rff']['l_vec'],
            seed=settings['seed_no']
        )
        model_time = time.time() - model_start
        print("Model training time:", model_time, "seconds")
        op[val_end] = model_op
    inner_time = time.time() - inner_start
    print("Total time for current horizon:", inner_time, "seconds")

    # Gem model-output for den aktuelle horizon til en pickle-fil (svarer til R's saveRDS)
    model_filename = f"{output_path}/model_{h}.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(op, f)

    models.append(op)

total_time = time.time() - start_time
print("Total run time:", total_time, "seconds")

Barra cov

In [ ]:
# Hvis du ikke har brug for den fjerde returnerede værdi, kan du bruge en underscore
cluster_data_d, ind_factors, clusters, cluster_data_m  = Estimate_Covariance_Matrix.process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)

fct_ret_est = Estimate_Covariance_Matrix.run_regressions_by_date(cluster_data_d, ind_factors, clusters)
fct_ret = Estimate_Covariance_Matrix.create_factor_returns(fct_ret_est)

w_cor = (0.5 ** (1 / settings['cov_set']['hl_cor'])) ** np.arange(settings['cov_set']['obs'], 0, -1)
w_var = (0.5 ** (1 / settings['cov_set']['hl_var'])) ** np.arange(settings['cov_set']['obs'], 0, -1)

fct_dates = np.sort(fct_ret["date"].unique())
min_calc_date = fct_dates[settings["cov_set"]["obs"] - 1] - pd.DateOffset(months=1)
calc_dates = np.sort(cluster_data_m.loc[cluster_data_m["eom"] >= min_calc_date, "eom"].unique())

factor_cov_est = Estimate_Covariance_Matrix.factor_cov_estimate(calc_dates, fct_dates, fct_ret, w_cor, w_var, settings)

spec_risk = Estimate_Covariance_Matrix.unnest_spec_risk(fct_ret_est)
spec_risk_res_vol = Estimate_Covariance_Matrix.calculate_ewma(spec_risk, settings)

spec_risk_m, spec_risk_res_vol = Estimate_Covariance_Matrix.process_spec_risk_m(fct_dates, spec_risk_res_vol)
barra_cov = Estimate_Covariance_Matrix.calculate_barra_cov(calc_dates, cluster_data_m, spec_risk_m, factor_cov_est)

with open('./data_fifty/barra_cov.pkl', 'wb') as f:
    pickle.dump(barra_cov, f)